In [26]:
import pandas as pd
from tqdm import trange, tqdm
import numpy as np
import torch
import sys
from collections import defaultdict
sys.path.append('/mnt/nfs/zhangtl/utils/')
from util import myout
import pickle as pkl

import dgl

## load

In [27]:
papers = pd.read_csv('../../raw_data/reddit/soc-redditHyperlinks-title.tsv', sep='\t')
# papers.columns = ['source', 'target', 'rating', 'time']
# papers = papers.sort_values(by=['time', 'source', 'target'])
papers = papers.sort_values(by=['TIMESTAMP', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,..."
0,rddtgaming,rddtrust,1u4pzzs,2013-12-31 16:39:18,1,"25.0,23.0,0.76,0.0,0.44,0.12,0.12,4.0,4.0,0.0,..."
35,examplesofevil,pics,1u5lhas,2013-12-31 16:40:29,1,"96.0,83.0,0.833333333333,0.0,0.0625,0.14583333..."
26,languagelearning,linguistics,1u5dkls,2013-12-31 17:41:16,1,"132.0,115.0,0.810606060606,0.0,0.0530303030303..."
1,xboxone,battlefield_4,1u4tmfs,2013-12-31 17:59:11,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
...,...,...,...,...,...,...
571891,peoplewhosayheck,sex,68iffo,2017-04-30 16:36:07,-1,"116.0,100.0,0.724137931034,0.0172413793103,0.1..."
571923,peoplewhosayheck,spiderman,68iqh6s,2017-04-30 16:36:07,1,"121.0,105.0,0.719008264463,0.0165289256198,0.0..."
571902,peoplewhosayheck,teenmom,68ii97s,2017-04-30 16:36:07,1,"137.0,115.0,0.759124087591,0.0,0.0510948905109..."
571892,brasilonreddit,bad_cop_no_donut,68ifuvs,2017-04-30 16:47:50,1,"194.0,162.0,0.773195876289,0.0,0.0670103092784..."


In [28]:
papers['time'] = papers['TIMESTAMP'].apply(lambda u: int(''.join(u.split(' ')[0].split('-'))))
papers = papers.sort_values(by=['time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,...",20131231
9,bestof,confession,1u56b3s,2013-12-31 21:41:46,1,"41.0,37.0,0.80487804878,0.0,0.121951219512,0.1...",20131231
15,bitcoinmining,bitcoin,1u58kus,2013-12-31 23:17:46,1,"82.0,70.0,0.756097560976,0.0,0.268292682927,0....",20131231
5,cancer,fuckcancer,1u5216s,2013-12-31 18:59:26,1,"94.0,80.0,0.776595744681,0.0,0.106382978723,0....",20131231
228,circlejerkaustralia,australia,1u7vo8s,2013-12-31 18:42:38,1,"102.0,90.0,0.823529411765,0.0,0.598039215686,0...",20131231
...,...,...,...,...,...,...,...
566524,winniethepooh,art,671bobs,2017-04-30 08:32:26,1,"76.0,66.0,0.842105263158,0.0,0.0921052631579,0...",20170430
571706,womeninred,pics,68grmys,2017-04-30 10:21:17,1,"91.0,76.0,0.703296703297,0.0,0.0989010989011,0...",20170430
571606,worldofwarships,warships,68frpzs,2017-04-30 07:26:04,1,"43.0,38.0,0.767441860465,0.0,0.162790697674,0....",20170430
571640,xbox_jp,xboxone,68g1xms,2017-04-30 08:40:29,1,"30.0,28.0,0.833333333333,0.0333333333333,0.3,0...",20170430


In [29]:
myout(papers['time'].unique())

 : shape=(1217,), [20131231 20140101 20140102 ... 20170428 20170429 20170430]


## cut

In [31]:
cut_step = len(papers) / 40
cut_step

14298.175

In [32]:
cut_step = len(papers) / 40
papers['new_time'] = (papers.index.values / cut_step).astype(np.int)
papers = papers.sort_values(by=['new_time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

/tmp/ipykernel_3932196/3679045895.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  papers['new_time'] = (papers.index.values / cut_step).astype(np.int)


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time,new_time
7484,195,atheism,1w7830s,2014-01-26 08:57:24,1,"63.0,52.0,0.761904761905,0.0,0.111111111111,0....",20140126,0
5160,195,mugen,1vkd1is,2014-01-18 18:11:51,1,"101.0,86.0,0.712871287129,0.039603960396,0.148...",20140118,0
6982,19thworldproblems,askreddit,1w2oie,2014-01-24 15:27:39,-1,"98.0,85.0,0.816326530612,0.0,0.19387755102,0.1...",20140124,0
900,1dash1,gamedev,1ueztws,2015-07-13 15:42:02,1,"61.0,55.0,0.639344262295,0.114754098361,0.1311...",20150713,0
3929,2007scape,askreddit,1v8cwfs,2014-01-14 15:54:06,1,"39.0,36.0,0.846153846154,0.0,0.205128205128,0....",20140114,0
...,...,...,...,...,...,...,...,...
565376,zika_virus,science,66p81ts,2017-04-10 16:59:28,1,"99.0,83.0,0.79797979798,0.0,0.181818181818,0.1...",20170410,39
567219,znation,redditgetsdrawn,679x1rs,2017-04-25 02:29:13,1,"136.0,111.0,0.764705882353,0.0,0.0882352941176...",20170425,39
560221,zootopia,zistopia,653k1ws,2017-04-12 22:08:03,1,"39.0,36.0,0.794871794872,0.0,0.230769230769,0....",20170412,39
559058,zubergoodstories,writingprompts,64qvces,2017-04-09 10:21:53,1,"54.0,48.0,0.796296296296,0.0,0.166666666667,0....",20170409,39


In [33]:
for ii in range(0, 40):
    print(ii, len(np.where(papers['new_time'].to_numpy()==ii)[0]))

0 14299
1 14298
2 14298
3 14298
4 14298
5 14299
6 14298
7 14298
8 14298
9 14298
10 14298
11 14299
12 14298
13 14298
14 14298
15 14298
16 14298
17 14299
18 14298
19 14298
20 14298
21 14298
22 14299
23 14298
24 14298
25 14298
26 14298
27 14298
28 14299
29 14298
30 14298
31 14298
32 14298
33 14298
34 14299
35 14298
36 14298
37 14298
38 14298
39 14298


## build graph

In [34]:
feat = [float(item) for item in papers['PROPERTIES'].iloc[0].split(',')]
myout(feat)

feat : len=86, list([63.0, 52.0, 0.761904761905, ..., 0.0, 0.07142857142857142, 0.0])


In [35]:
def update_idx(idx, dic, cnt, feats, feat_dim, no_emb, ff):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        feats.append(torch.tensor(ff).to(torch.float32))
    return dic, cnt, feats, no_emb

In [36]:
start_year, end_year = 0, 40
feat_dim = 128
index = {}

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
last_len, last_year = 0, start_year

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    if year != last_year: # 进入下一个年份，则更新上个年份
        index[last_year] = [len(feats), last_len, len(lst)] # 记录当前年份的节点数、eind的起始和结束
        last_len = len(lst)
        last_year = year

    source = str(papers['SOURCE_SUBREDDIT'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    weight = int(papers['LINK_SENTIMENT'].iloc[ii])
    ff = [float(item) for item in papers['PROPERTIES'].iloc[ii].split(',')]
    
    id2nid, cnt, feats, no_emb = update_idx(source, id2nid, cnt, feats, feat_dim, no_emb, ff)
    id2nid, cnt, feats, no_emb = update_idx(target, id2nid, cnt, feats, feat_dim, no_emb, ff)
    
    lst.append((id2nid[source], id2nid[target], weight))

index[year] = [len(feats), last_len, len(lst)]
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
eind = torch.stack([src, tgt])

flag1, flag2 = '' in id2nid, np.nan in id2nid
mm = torch.max(eind).item()
print(f'feat:{feat.shape}, eind:{eind.shape},{mm}, rel:{len(rel)}, {flag1},{flag2}, {no_emb}')
inputs = {'feat': feat, 'eind': eind, 'rel': rel, 'id2nid': id2nid, 'index': index}
myout(feat, eind, rel, id2nid)

100%|██████████| 571927/571927 [00:24<00:00, 23821.66it/s]


feat:torch.Size([54075, 86]), eind:torch.Size([2, 571927]),54074, rel:571927, False,False, 0
feat : shape=torch.Size([54075, 86])
tensor([[6.3000e+01, 5.2000e+01, 7.6190e-01,  ..., 0.0000e+00, 7.1429e-02,
         0.0000e+00],
        [6.3000e+01, 5.2000e+01, 7.6190e-01,  ..., 0.0000e+00, 7.1429e-02,
         0.0000e+00],
        [1.0100e+02, 8.6000e+01, 7.1287e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [4.3000e+01, 3.6000e+01, 7.2093e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [9.9000e+01, 8.3000e+01, 7.9798e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.9000e+01, 3.6000e+01, 7.9487e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])
eind : shape=torch.Size([2, 571927])
tensor([[    0,     0,     3,  ..., 37902, 53180, 26909],
        [    1,     2,     4,  ..., 54074,   699, 10164]])
rel : shape=torch.Size([571927]), tensor([ 1,  1, -1,  ...,  1,  1,  1])
id2nid : len=54075, dict([195: 0, atheism: 1, m

In [37]:
index

{0: [5195, 0, 14299],
 1: [7933, 14299, 28597],
 2: [10046, 28597, 42895],
 3: [11907, 42895, 57193],
 4: [13580, 57193, 71491],
 5: [15147, 71491, 85790],
 6: [16587, 85790, 100088],
 7: [17919, 100088, 114386],
 8: [19290, 114386, 128684],
 9: [20633, 128684, 142982],
 10: [21925, 142982, 157280],
 11: [23234, 157280, 171579],
 12: [24544, 171579, 185877],
 13: [25709, 185877, 200175],
 14: [26910, 200175, 214473],
 15: [28233, 214473, 228771],
 16: [29510, 228771, 243069],
 17: [30714, 243069, 257368],
 18: [31860, 257368, 271666],
 19: [32987, 271666, 285964],
 20: [34020, 285964, 300262],
 21: [35153, 300262, 314560],
 22: [36340, 314560, 328859],
 23: [37464, 328859, 343157],
 24: [38820, 343157, 357455],
 25: [39934, 357455, 371753],
 26: [41074, 371753, 386051],
 27: [42127, 386051, 400349],
 28: [43331, 400349, 414648],
 29: [44360, 414648, 428946],
 30: [45332, 428946, 443244],
 31: [46338, 443244, 457542],
 32: [47288, 457542, 471840],
 33: [48246, 471840, 486138],
 34: [491

In [38]:
pkl.dump(inputs, open(f'../../../01_process/data_papers/rdt/inputs.pkl', 'wb'))

## labels

In [39]:
start_year, end_year = 0, 40
cites = {}
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    cites[year][target] += 1
myout(cites[5])

100%|██████████| 571927/571927 [00:07<00:00, 78969.11it/s]

 : len=2920, dict([australia: 22, guns: 15, aviation: 3, showerthoughts: 72, harley: 2, usmc: 3, ...])


In [40]:
labels = {}
id2nid, index = inputs['id2nid'], inputs['index']
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = index[year][1:]
    eind = inputs['eind'][:, left:right]
    
    nids = torch.unique(eind[0, :]).tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf

print(len(labels))

38: 100%|██████████| 1/1 [00:00<00:00, 164.97it/s, pdf=4496, year=38]
39: 0it [00:00, ?it/s]


40


In [41]:
labels[5]

,id,nid,6,7,8,9,10,11,12,13,...,30,31,32,33,34,35,36,37,38,39
0,195,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,3.0,0.0
1,atheism,1,63.0,54.0,50.0,62.0,42.0,50.0,49.0,66.0,...,23.0,36.0,45.0,54.0,42.0,24.0,38.0,42.0,25.0,26.0
2,2007scape,7,3.0,5.0,5.0,8.0,5.0,9.0,6.0,5.0,...,12.0,10.0,11.0,18.0,23.0,4.0,8.0,21.0,13.0,15.0
3,gaming,8,98.0,114.0,133.0,104.0,103.0,118.0,140.0,110.0,...,94.0,122.0,128.0,127.0,111.0,129.0,109.0,127.0,98.0,83.0
4,2014elections,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,yorkcountypa,15138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4026,yuruyuri,15141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4027,zazie_lavender,15142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4028,zioncc,15145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
pkl.dump(labels, open(f'../../../01_process/data_papers/rdt/labels.pkl', 'wb'))

In [43]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-1):
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[5]

39


,id,nid,6,7,8,9,10,11,12,13,...,30,31,32,33,34,35,36,37,38,39
0,195,0,0.000000,0.693147,0.693147,0.693147,0.693147,1.098612,1.098612,1.098612,...,1.945910,2.079442,2.197225,2.302585,2.302585,2.484907,2.639057,2.639057,2.833213,2.833213
1,atheism,1,4.158883,4.770685,5.123964,5.438079,5.605802,5.774551,5.916202,6.079933,...,7.074117,7.104144,7.140453,7.182352,7.213768,7.231287,7.258412,7.287560,7.304516,7.321850
2,2007scape,7,1.386294,2.197225,2.639057,3.091043,3.295837,3.583519,3.737670,3.850148,...,5.505332,5.545177,5.587249,5.652489,5.730100,5.743003,5.768321,5.831882,5.869297,5.910797
3,gaming,8,4.595120,5.361292,5.846439,6.109248,6.315358,6.508769,6.698268,6.825460,...,7.925880,7.969012,8.012350,8.053569,8.088255,8.127110,8.158802,8.194506,8.221210,8.243282
4,2014elections,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,yorkcountypa,15138,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
4026,yuruyuri,15141,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294
4027,zazie_lavender,15142,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4028,zioncc,15145,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
pkl.dump(labels_cum_log, open(f'../../../01_process/data_papers/rdt/labels_cum_log.pkl', 'wb'))